In [1]:
import pandas as pd
import pickle

from utils_mitgcm import open_mitgcm_ds_from_config

In [2]:
model = 'geneva_200m'
params_file = 'swirl_03'

# Open MITgcm results

In [3]:
mitgcm_config, ds_mitgcm = open_mitgcm_ds_from_config('../config.json', model)

In [ ]:
with open(rf'../Outputs/{model}_{params_file}_eddies.pkl', 'rb') as f:
    eddies = pickle.load(f)

In [ ]:
def compute_ke_snapshot(snapshot_vel):
    ke = 0.5 * (snapshot_vel.UVEL.values**2 + snapshot_vel.VVEL.values**2 + snapshot_vel.WVEL.values**2) * snapshot_vel.dxC.values * snapshot_vel.dyC.values * snapshot_vel.drC.values  # This gives J per cell
    
    return ke / 1e6  # Convert to MJ

In [ ]:
def translate_rotation_direction(eddy_orientation: int):
    return 'clockwise' if eddy_orientation == -1 else 'anticlockwise' 

In [ ]:
df_catalogue_level0 = pd.DataFrame(columns=[
    'id',
    'date',
    'depth_[m]',
    'xc',
    'yc',
    'surface_area_[m2]',
    'volume_slice_[m3]',
    'rotation_direction',
    'kinetic_energy_[MJ]'
])
id_level0 = 0
for t_index in eddies['time_indices']:
    for d_index in eddies['depth_indices']:
        snapshot_vel = ds_mitgcm.isel(time=t_index, Z=d_index, Zp1=d_index, Zl=d_index, Zu=d_index)
        snapshot_eddies = eddies[t_index][d_index]
        surface_cell = snapshot_vel.dxC.values[0][0] * snapshot_vel.dyC.values[0][0]
        ke_snapshot_megajoules = compute_ke_snapshot(snapshot_vel)
        for eddy in snapshot_eddies:
            print(f'Processing eddy {id_level0}')
            vortex_indices = tuple(eddy.vortex_cells.astype(int))
            ke_eddy = ke_snapshot_megajoules[vortex_indices[1], vortex_indices[0]].sum()
            surface_area = len(eddy.vortex_cells[0]) * surface_cell
            
            df_catalogue_level0.loc[id_level0] = [
                id_level0,                                                       # int
                pd.Timestamp(snapshot_vel.time.values).to_pydatetime(),          # datetime64 -> datetime.datetime
                float(snapshot_vel.Z.values),                                    # float
                eddy.center[0],                                                  # tuple
                eddy.center[1],
                float(surface_area),                                             # numpy.float32 -> float
                float(surface_area * float(snapshot_vel.drC.values)),            # volume
                translate_rotation_direction(eddy.orientation),                  # str
                float(ke_eddy)                                                   # numpy.float64 -> float
            ]
            id_level0 += 1